<a href="https://colab.research.google.com/github/gladcolor/tweet_translator/blob/master/tweet_translator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Multi-language translation.

Please refer to [Beyond English-Centric Multilingual Machine Translation](https://arxiv.org/abs/2010.11125) and [Hugginface](https://huggingface.co/transformers/model_doc/m2m_100.html#overview).

# Install package

In [ ]:
! pip install sentencepiece
! pip install transformers==4.6.0
! pip install googletrans==4.0.0rc1

     |████████████████████████████████| 1.2MB 8.7MB/s 
     |████████████████████████████████| 2.3MB 9.0MB/s 
     |████████████████████████████████| 3.3MB 58.6MB/s 
     |████████████████████████████████| 901kB 50.6MB/s 
     |████████████████████████████████| 61kB 5.2MB/s 
     |████████████████████████████████| 1.0MB 16.2MB/s 
     |████████████████████████████████| 51kB 7.4MB/s 
     |████████████████████████████████| 71kB 9.4MB/s 
     |████████████████████████████████| 61kB 8.3MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-cp37-none-any.whl size=17417 sha256=5dbd379f9a2ac1ca19c598152f3849e6771ef475214f1a0944932c0148562793
  Stored in directory: /root/.cache/pip/wheels/09/32/56/fd8940f1b3c1d77c9f91b55597c52a4d4833b000a980bb0740
Successfully built googletrans


# Import package

In [ ]:
import os
import pandas as pd
from googletrans import Translator

from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
google_trans = Translator()

# Load pretrained model

In [ ]:
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to("cuda:0")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_1.2B").to("cuda:0")
# tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_1.2B")

# Translate some texts (English to Chinese, Chinese to English)

In [ ]:
def trans(raw_text, out_lang = 'en'):
    encoded_text = tokenizer(raw_text, return_tensors="pt").to("cuda:0")
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(out_lang))
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return result

def main():

    in_lang = 'en'
    out_lang = 'zh'
    tokenizer.src_lang = in_lang
    tweets_csv = r"https://raw.githubusercontent.com/gladcolor/tweet_translator/master/tweets_samples.zip"
    
    tweets_df = pd.read_csv(tweets_csv, encoding='ISO-8859-1')
    tweets_df = tweets_df.sample(100)
    for idx, row in tweets_df.iterrows():
        raw_text = row["text"]
        result = trans(raw_text, out_lang=out_lang)
        print("Raw text:       ", raw_text)        
        print("Translated text:", result[0])
        print()


    in_lang = 'zh'
    out_lang = 'en'
    tokenizer.src_lang = in_lang
    tweets_csv = r"https://raw.githubusercontent.com/gladcolor/tweet_translator/master/food_for_thought.zip"
    
    tweets_df = pd.read_csv(tweets_csv)
    # tweets_df = tweets_df.sample(10)
    for idx, row in tweets_df.iterrows():
        raw_text = row["text"]
        result = trans(raw_text, out_lang=out_lang)
        print("Raw text:       ", raw_text)        
        print("Translated text:", result[0])
        print()


main()

Raw text:        Health is more priority than anything in this world so pls govt postpone the exams until the COVIND 19 VACCINE is developed..
#CancelExamsSaveLives
Translated text: 健康是这个世界上任何东西的优先事项,所以 pls govt 推迟考试,直到COVIND 19 疫苗开发。

Raw text:        Center: UPHC Motikatla MHA
District: Jaipur I
Pincode: 302004
Type: Free
Date: 01-06-2021
Min Age: 45
Vaccine: COVISHIELD
Dose1: 5
Dose2: 0

Telegram: https://t.co/cBSgxuLt65

#jaipur #rajasthan #covid19 #vaccine #slot #covishield #covaxin #SputnikV
[20:18]
Translated text: 中心: UPHC Motikatla MHA 区: Jaipur I Pincode: 302004 类型: 免费日期: 01-06-2021 年龄: 45 疫苗: COVISHIELD 剂量1: 5 剂量2: 0 电报: https://t.co/cBSgxuLt65 #jaipur #rajasthan #covid19 #疫苗 #slot #covishield #covaxin #SputnikV [20:18]

Raw text:        Me explaining the importance of  vaccination to friends and family! ????

Also me after getting vaccinated! ????

Got my first shot of vaccine! ???? ????
#COVID19Vaccination
#Vaccinateindia
#Covishield https://t.co/ubb3SZF3jr
Translated text

# Translate a single sentence. 

Note: The model cannot translate long paragraph.

In [ ]:
def trans_with_tokenizer(raw_text, in_lang='zh', out_lang = 'en'):
    tokenizer.src_lang = in_lang
    encoded_text = tokenizer(raw_text, return_tensors="pt").to("cuda:0")
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(out_lang))
    result = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

    print("Raw text:       ", raw_text)        
    print("Translated text:", result[0])
    print()

    return result

In [ ]:
raw_text = '父亲的生命已走到了尽头，他用尽力气呼吸，比他在井下扛起二百多斤的铁支架时用的力气大得多。他的脸惨白，双目突出，嘴唇因窒息而呈深紫色，仿佛一条无形的绞索正在脖子上慢慢绞紧，他那辛劳一生的所有淳朴的希望和梦想都已消失，现在他生命的全部渴望就是多吸进一点点空气。但父亲的肺，就像所有患三期矽肺病的矿工的肺一样，成了一块由网状纤维连在一起的黑色的灰块，再也无法把吸进的氧气输送到血液中。组成那个灰块的煤粉是父亲在二十五年中从井下一点点吸入的，这也证明他一生采出的煤有多大的量了。'

trans_with_tokenizer(raw_text, in_lang='zh', out_lang='en')

Raw text:        父亲的生命已走到了尽头，他用尽力气呼吸，比他在井下扛起二百多斤的铁支架时用的力气大得多。他的脸惨白，双目突出，嘴唇因窒息而呈深紫色，仿佛一条无形的绞索正在脖子上慢慢绞紧，他那辛劳一生的所有淳朴的希望和梦想都已消失，现在他生命的全部渴望就是多吸进一点点空气。但父亲的肺，就像所有患三期矽肺病的矿工的肺一样，成了一块由网状纤维连在一起的黑色的灰块，再也无法把吸进的氧气输送到血液中。组成那个灰块的煤粉是父亲在二十五年中从井下一点点吸入的，这也证明他一生采出的煤有多大的量了。
Translated text: His father’s life had come to an end, and he breathed as hard as he could, much more than when he carried over two hundred kilograms of iron shelves under the well. His face was white, his eyes highlighted, his lips were deep purple because of suffocation, as if an invisible hanging was slowly hanging on his neck, all the simple hopes and dreams of his hard life had disappeared, and now his whole desire for life was to breathe in a little more air. But his father’s lungs, like the lungs of all the miners who suffered from silicon pneumonia, became a black gray piece connected by retinal fibers, and then transported the oxygen that could not be absorbed into the blood.



['His father’s life had come to an end, and he breathed as hard as he could, much more than when he carried over two hundred kilograms of iron shelves under the well. His face was white, his eyes highlighted, his lips were deep purple because of suffocation, as if an invisible hanging was slowly hanging on his neck, all the simple hopes and dreams of his hard life had disappeared, and now his whole desire for life was to breathe in a little more air. But his father’s lungs, like the lungs of all the miners who suffered from silicon pneumonia, became a black gray piece connected by retinal fibers, and then transported the oxygen that could not be absorbed into the blood.']

In [ ]:
raw_text = '今天（6月11日），国家航天局举行天问一号探测器着陆火星首批科学影像图揭幕仪式，公布了由“祝融号”火星车拍摄的着陆点全景、火星地形地貌、“中国印迹”和“着巡合影”等影像图。首批科学影像图的发布，标志着我国首次火星探测任务取得圆满成功。'

trans_with_tokenizer(raw_text, in_lang='zh', out_lang='en')

Raw text:        今天（6月11日），国家航天局举行天问一号探测器着陆火星首批科学影像图揭幕仪式，公布了由“祝融号”火星车拍摄的着陆点全景、火星地形地貌、“中国印迹”和“着巡合影”等影像图。首批科学影像图的发布，标志着我国首次火星探测任务取得圆满成功。
Translated text: Today (June 11), the National Space Agency held the opening ceremony of the landing of the first scientific imaging maps of the Mars spacecraft, announced the landing point panorama, the landscape of Mars, "China imprints" and "Cruise Images" and other imaging maps photographed by the "Hi Fusion" Mars vehicle. the release of the first scientific imaging images, marks the success of China's first Mars exploration mission.



In [ ]:
raw_text = 'China’s famed wandering elephants are on the move again, heading southwest while a male who broke from the herd is still keeping his distance.The group left a wildlife reserve in the southwest of Yunnan province more than a year ago and has trekked 500 kilometers (300 miles) north to the outskirts of the provincial capital of Kunming.'

result = trans_with_tokenizer(raw_text, in_lang='en', out_lang='zh')
result_back = trans_with_tokenizer(result, in_lang='zh', out_lang='en')

Raw text:        China’s famed wandering elephants are on the move again, heading southwest while a male who broke from the herd is still keeping his distance.The group left a wildlife reserve in the southwest of Yunnan province more than a year ago and has trekked 500 kilometers (300 miles) north to the outskirts of the provincial capital of Kunming.
Translated text: 中国著名的流浪大象再次移动,向西南方向,而一个从群中脱离的雄性仍然保持距离,该团体在一年多前离开了云南省西南部的一个野生动物保护区,并向北走了500公里到省首都昆明的边缘。

Raw text:        ['中国著名的流浪大象再次移动,向西南方向,而一个从群中脱离的雄性仍然保持距离,该团体在一年多前离开了云南省西南部的一个野生动物保护区,并向北走了500公里到省首都昆明的边缘。']
Translated text: China’s famous wandering elephant moves southwest again, while a male separated from the herd still keeps a distance, leaving a wildlife reserve in southwestern Yunnan province more than a year ago and walking 500 kilometers north to the edge of the province’s capital, Kunming.



# Compare with Google Translation

In [ ]:
raw_text = '在福建中部的阳春村，村民每年都要聚集在宗祠举行仪式，向他们供奉的佛像——章公祖师祈福。其实，村民祭拜的，只是佛像的复制品。真品早已不见。 阳春村隶属三明市大田县吴山镇。自北宋起，这个村和东埔村共同拥有的林氏宗祠普照堂内，章公祖师的肉身佛像曾端坐千年。1995年12月，一桩盗窃案发生后，佛像下落不明。此后20年，杳无音信。'
result = trans_with_tokenizer(raw_text, in_lang='zh', out_lang='en')
result_back = trans_with_tokenizer(result, in_lang='en', out_lang='zh')

g_result = google_trans.translate(raw_text, dest='en').text
print("Raw text:       ", raw_text)        
print("Google Translated:", g_result)
print()


raw_text = g_result
g_result = google_trans.translate(raw_text, dest='zh-cn').text
print("Raw text:       ", raw_text)        
print("Google Translated:", g_result)
print()

Raw text:        在福建中部的阳春村，村民每年都要聚集在宗祠举行仪式，向他们供奉的佛像——章公祖师祈福。其实，村民祭拜的，只是佛像的复制品。真品早已不见。 阳春村隶属三明市大田县吴山镇。自北宋起，这个村和东埔村共同拥有的林氏宗祠普照堂内，章公祖师的肉身佛像曾端坐千年。1995年12月，一桩盗窃案发生后，佛像下落不明。此后20年，杳无音信。
Translated text: In Yangzhou village in the middle of Fujian, the villagers gather every year at the temple to hold a ceremony to worship the Buddha statue of their worship—the Buddha monk. In fact, the villagers worshiped only a copy of the Buddha statue. The real property has long been lost. Yangzhou village belongs to the Sanming city of Oda County Wu Mountain Town. Since the northern Song, the village and the Dongbo village jointly owned the Buddha statue of the Buddha, the Buddha's body has been seated for thousands of years. In December 1995, after a theft, the Buddha statue disappeared.

Raw text:        ["In Yangzhou village in the middle of Fujian, the villagers gather every year at the temple to hold a ceremony to worship the Buddha statue of their worship—the Buddha monk. In fact, the villagers wors

# Test the reverse translation. (English->Chinese->English->Chinese)

In [ ]:
raw_text = r'“Do Millennials in China also love avocado toast?”A global executive asked me this, in earnest, a month ago. I get questions like this because I am the founder of a market-insights company called Young China Group. Whenever I give talks, whether it’s London or Cairo or Rio, people always ask the same question: “How do the younger generations in China stack up against our own?”'
g_result = google_trans.translate(raw_text, dest='zh-cn').text
print("Raw text:       ", raw_text)        
print("Google Translated:", g_result)
print()

result = trans_with_tokenizer(raw_text, in_lang='en', out_lang='zh')


result_back = trans_with_tokenizer(result, in_lang='zh', out_lang='en')
result_back = trans_with_tokenizer(result_back, in_lang='en', out_lang='zh')
result_back = trans_with_tokenizer(result, in_lang='zh', out_lang='en')
result_back = trans_with_tokenizer(result_back, in_lang='en', out_lang='zh')

Raw text:        “Do Millennials in China also love avocado toast?”A global executive asked me this, in earnest, a month ago. I get questions like this because I am the founder of a market-insights company called Young China Group. Whenever I give talks, whether it’s London or Cairo or Rio, people always ask the same question: “How do the younger generations in China stack up against our own?”
Google Translated: “在中国做千禧一代也喜欢鳄梨吐司？”一个全球主管在一个月前恳切地问我这件事。我得到这样的问题，因为我是一个名为年轻中国集团的市场洞察公司的创始人。每当我谈话时，无论是伦敦还是开罗或里约热内卢，人们总是问同样的问题：“中国的年轻世代在我们自己堆叠？”

Raw text:        “Do Millennials in China also love avocado toast?”A global executive asked me this, in earnest, a month ago. I get questions like this because I am the founder of a market-insights company called Young China Group. Whenever I give talks, whether it’s London or Cairo or Rio, people always ask the same question: “How do the younger generations in China stack up against our own?”
Translated text: “中国千禧一代也喜欢喝阿沃卡多吗?”一个全球性高管在一个月前认真地问我这个问题,我会得到